In [1]:
#Problem: The Google of Quotes
#Group members: S. Wang, Y. Ye, M. Staton, W. Wang

In [2]:
import math, re
from collections import Counter

In [3]:
quotes_file = open("quotes.txt", "r") #make a handle on the file

In [4]:
quote_list = []

new_line = quotes_file.readline()

while len(new_line) != 0:
    
    str1 = new_line.strip("\r\n")
    
    next_line = quotes_file.readline()
    
    str2 = next_line.strip("\r\n")
        
    quote_list.extend([" - ".join([str1, str2])])
    
    new_line = quotes_file.readline()

In [5]:
quotes_file.close()

In [6]:
def words_in_quote(quote):
    
    pattern = "[^a-zA-Z0-9_]+"
    
    word_list = [word.lower() for word in re.split(pattern, quote)]
    
    while "" in word_list:
        
        word_list.remove("")
        
    return word_list

In [20]:
def count_ocur(obj, elem):
    
    count = 0
    
    if type(obj) == list:
        
        for i in range(0, len(obj)):

            if str(obj[i]).lower() == elem.lower():

                count += 1

        return count
    
    elif type(obj) == str:
        
        obj_list = re.split("[^a-zA-Z0-9_]+", obj.lower())
        
        for j in range(0, len(obj_list)):
            
            if obj_list[j] == elem:
                
                count += 1
                
        return count
    
    else:
        
        return 0

In [21]:
postings_list_dict = {}

total_words = []

for quote in quote_list:
        
    word_list = words_in_quote(quote)
    
    unique_words = Counter(word_list).keys()
    
    total_words.extend(unique_words)
    
    value_dict = {}
    
    for word in unique_words:
        
        value_dict[word] = count_ocur(word_list, word)
        
    postings_list_dict[quote] = value_dict

In [22]:
type(word_list) == list

True

In [23]:
postings_list_dict

{'.I had to face the horrible truth: The antitobacco people are lying. Smoking really is cool. And I`m less cool for not doing it. - Tucker Carlson': {'and': 1,
  'antitobacco': 1,
  'are': 1,
  'carlson': 1,
  'cool': 2,
  'doing': 1,
  'face': 1,
  'for': 1,
  'had': 1,
  'horrible': 1,
  'i': 2,
  'is': 1,
  'it': 1,
  'less': 1,
  'lying': 1,
  'm': 1,
  'not': 1,
  'people': 1,
  'really': 1,
  'smoking': 1,
  'the': 2,
  'to': 1,
  'truth': 1,
  'tucker': 1},
 'A Native American elder once described his own inner struggles in this manner: Inside of me there are two dogs. One of the dogs is mean and evil. The other dog is good. The mean dog fights the good dog all the time. When asked which dog wins, he reflected. - George Bernard Shaw': {'a': 1,
  'all': 1,
  'american': 1,
  'and': 1,
  'are': 1,
  'asked': 1,
  'bernard': 1,
  'described': 1,
  'dog': 4,
  'dogs': 2,
  'elder': 1,
  'evil': 1,
  'fights': 1,
  'george': 1,
  'good': 2,
  'he': 1,
  'his': 1,
  'in': 1,
  'inner

In [24]:
rev_posting_list_dict = {}

total_unique_words = Counter(total_words).keys()

for word in total_unique_words:
    
    value_dict2 = {}
    
    for quote in quote_list:
        
        value_dict2[quote] = count_ocur(quote, word)
        
    rev_posting_list_dict[word] = value_dict2

In [25]:
def tf_idf((quote, word)):
    
    word = str(word).lower()
    
    if quote not in postings_list_dict:
        
        print "The quote is not found."
        
        return 0
    
    if word not in rev_posting_list_dict:
                
        print "The word is not found."
        
        return 0
    
    max_ocur_in_quote = max([value for (key, value) in postings_list_dict[quote].items() ])
        
    TF = float(count_ocur(quote, word)) / max_ocur_in_quote
        
    N = len(quote_list)
        
    n = len([(key, value) for (key, value) in rev_posting_list_dict[word].items() if value > 0])
        
    IDF = math.log(float(N/n))
        
    TF_IDF = TF * IDF
        
    return TF_IDF

In [26]:
[value for (key, value) in postings_list_dict[quote_list[16]].items() ]

[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2]

In [27]:
print tf_idf((quote_list[16], "entertainer"))

3.3933584753


In [32]:
def sg_word_search(word):
    
    word = str(word).lower()
        
    search_dict = {}
    
    if word not in rev_posting_list_dict:
        
        print word, " is not found."
        
        return {}
        
    retrieved_quotes = [key for (key, value) in rev_posting_list_dict[word].items() if value > 0]
    
    for quote in retrieved_quotes:
        
        search_dict[quote] = tf_idf((quote, word))
        
        #print word, " in ", quote, word in quote
    
    return search_dict

In [29]:
def merge_lists(lst):
    
    new_lst = []
    
    for elem in lst:
        
        new_lst += lst
        
    return new_lst

In [30]:
def mul_word_search(words):
    
    words = [str(s).lower() for s in words]
    
    search_dict2 = {}
    
    retrieved_quotes2_unmerged = map(sg_word_search, words)
    
    retrieved_quotes2_merged = []
    
    for i in range(0, len(retrieved_quotes2_unmerged)):
        
        retrieved_quotes2_merged += retrieved_quotes2_unmerged[i]
    
    retrieved_quotes2_unique = Counter(retrieved_quotes2_merged).keys()
        
    for quote in retrieved_quotes2_unique:
        
        new_params = [(quote, word) for word in words]
        
        search_dict2[quote] = sum(map(tf_idf, new_params))
        
    return search_dict2

In [33]:
len(sg_word_search("and"))

269

In [34]:
mul_word_search(["And", "or"])

{'.I had to face the horrible truth: The antitobacco people are lying. Smoking really is cool. And I`m less cool for not doing it. - Tucker Carlson': 0.5493061443340549,
 'A Native American elder once described his own inner struggles in this manner: Inside of me there are two dogs. One of the dogs is mean and evil. The other dog is good. The mean dog fights the good dog all the time. When asked which dog wins, he reflected. - George Bernard Shaw': 0.21972245773362198,
 'A University of Chicago professor said the greatest day in the world was when a water puppy crawled out on land and decided to stay. The water puppy, he said, went on to become a man. If he proves that, I am willing to give up Christmas, Thanksgiving, and New Year`s Day, and to celebrate Water Puppy Day. - William Jennings Bryan': 0.8239592165010823,
 'A few hours of mountain climbing turn a villain and a saint into two rather equal creature. Exhaustion is the shortest way to equality and fraternity, and liberty is add